In [1]:
from constants import JQL, FIELDS, BASE_URL, JIRA_DOMAIN,EMAIL, MAX_RESULTS, MODULE_DEVS, VALID_STATUSES, MAIL_MAP, DAILY_HOURS,MIN_PROJECT_RATIO,PROJECT_MAP, DEFAULT_END_DATE, DEFAULT_END_DATE_with_timezone, DEFAULT_START_DATE,DEFAULT_START_DATE_with_timezone
from token_hidden import API_TOKEN



In [2]:
import requests
import json
from requests.auth import HTTPBasicAuth

HEADERS = {
    "Accept": "application/json"
}
AUTH = HTTPBasicAuth(EMAIL, API_TOKEN)

# Pagination loop
start_at = 0
all_issues = []

while True:
    params = {
        "jql": JQL,
        "maxResults": MAX_RESULTS,
        "startAt": start_at,
        "fields": FIELDS
    }

    response = requests.get(BASE_URL, headers=HEADERS, params=params, auth=AUTH)
    response.raise_for_status()
    page_data = response.json()

    issues = page_data.get("issues", [])
    all_issues.extend(issues)

    print(f"🔄 Fetched {len(issues)} issues (startAt={start_at})")

    if len(issues) < MAX_RESULTS:
        break  # no more pages
    start_at += MAX_RESULTS

# Store full dataset
with open("jira_it_issues.json", "w", encoding="utf-8") as f:
    json.dump({"issues": all_issues}, f, indent=2, ensure_ascii=False)

print(f"✅ Total issues saved: {len(all_issues)}")

### Download epics
def fetch_epics():
    start_at = 0
    all_epics = {}

    while True:
        params = {
            "jql": "project = IT AND issuetype = Epic",
            "maxResults": 100,
            "startAt": start_at,
            "fields": "key,duedate,summary"
        }

        response = requests.get(BASE_URL, headers=HEADERS, params=params, auth=AUTH)
        response.raise_for_status()
        data = response.json()

        for issue in data["issues"]:
            epic_key = issue["key"]
            due = issue["fields"].get("duedate")
            summary = issue["fields"].get("summary", "")
            
            # Inicializar con datos del epic
            all_epics[epic_key] = {
                "due_date": due,
                "summary": summary,
                "tasks": []
            }

            # 🔄 Buscar tareas asociadas a la épica
            story_start = 0
            while True:
                story_params = {
                    "jql": f'"Epic Link" = {epic_key}',
                    "maxResults": 100,
                    "startAt": story_start,
                    "fields": "key,customfield_10016,status"
                }

                story_resp = requests.get(BASE_URL, headers=HEADERS, params=story_params, auth=AUTH)
                story_resp.raise_for_status()
                story_data = story_resp.json()

                for story in story_data["issues"]:
                    story_key = story["key"]
                
                    sp_estimate = story["fields"].get("customfield_10016")  #  Story Points
                    status = story["fields"].get("status", {}).get("name", "Sin estado")
                    all_epics[epic_key]["tasks"].append({
                        "key": story_key,
                        "story_points": sp_estimate,
                        "status": status
                    })

                if len(story_data["issues"]) < 100:
                    break
                story_start += 100

        if len(data["issues"]) < 100:
            break
        start_at += 100

    return all_epics

epics = fetch_epics()

# Guardar en archivo si lo deseas
with open("epics_due_lookup.json", "w") as f:
    json.dump(epics, f, indent=2)

🔄 Fetched 100 issues (startAt=0)
🔄 Fetched 100 issues (startAt=100)
🔄 Fetched 100 issues (startAt=200)
🔄 Fetched 100 issues (startAt=300)
🔄 Fetched 100 issues (startAt=400)
🔄 Fetched 100 issues (startAt=500)
🔄 Fetched 100 issues (startAt=600)
🔄 Fetched 100 issues (startAt=700)
🔄 Fetched 100 issues (startAt=800)
🔄 Fetched 100 issues (startAt=900)
🔄 Fetched 100 issues (startAt=1000)
🔄 Fetched 100 issues (startAt=1100)
🔄 Fetched 100 issues (startAt=1200)
🔄 Fetched 100 issues (startAt=1300)
🔄 Fetched 100 issues (startAt=1400)
🔄 Fetched 100 issues (startAt=1500)
🔄 Fetched 100 issues (startAt=1600)
🔄 Fetched 100 issues (startAt=1700)
🔄 Fetched 100 issues (startAt=1800)
🔄 Fetched 100 issues (startAt=1900)
🔄 Fetched 100 issues (startAt=2000)
🔄 Fetched 100 issues (startAt=2100)
🔄 Fetched 100 issues (startAt=2200)
🔄 Fetched 100 issues (startAt=2300)
🔄 Fetched 100 issues (startAt=2400)
🔄 Fetched 100 issues (startAt=2500)
🔄 Fetched 100 issues (startAt=2600)
🔄 Fetched 100 issues (startAt=2700)
🔄 Fe

In [3]:
API_KEY = '346436483896-non6bmg405eh4avr5saql5m1r0r37rim.apps.googleusercontent.com'
client_secret = '346436483896-non6bmg405eh4avr5saql5m1r0r37rim.apps.googleusercontent.com'

In [4]:
import os
import datetime

from collections import defaultdict
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# 📌 Permisos necesarios: lectura del calendario
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# 📂 Archivos de autenticación
CREDENTIALS_PATH = 'client_secret.json'
TOKEN_PATH = 'token.json'



def obtener_credenciales():
    creds = None
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_PATH, 'w') as token:
            token.write(creds.to_json())
    return creds

def obtener_bloques_ocupados(email, creds, start_dt, end_dt):
    service = build('calendar', 'v3', credentials=creds)
    eventos_resultado = service.events().list(
        calendarId=email,
        timeMin=start_dt.isoformat(),
        timeMax=end_dt.isoformat(),
        singleEvents=True,
        orderBy="startTime"
    ).execute()

    eventos = eventos_resultado.get('items', [])
    bloques_ocupados = []

    for evento in eventos:
        start_str = evento['start'].get('dateTime')
        end_str = evento['end'].get('dateTime')
        if not start_str or not end_str:
            continue  # Omitir eventos de todo el día

        start_time = datetime.datetime.fromisoformat(start_str)
        end_time = datetime.datetime.fromisoformat(end_str)
        bloques_ocupados.append((start_time, end_time))

    return bloques_ocupados

def obtener_bloques_por_dev():
    creds = obtener_credenciales()
    bloques_por_dev = {}

    for dev, email in MAIL_MAP.items():
        print(f"🔎 Consultando reuniones de {dev} ({email})...")
        try:
            bloques = obtener_bloques_ocupados(email, creds, DEFAULT_START_DATE_with_timezone, DEFAULT_END_DATE_with_timezone)
            bloques_por_dev[dev] = bloques
            #print(f"   ✅ {len(bloques)} reuniones encontradas.\n")
        except Exception as e:
            print(f"   ❌ Error consultando {email}: {str(e)}\n")
            bloques_por_dev[dev] = []

    return bloques_por_dev

# Para usar de forma aislada:
if __name__ == "__main__":
    bloques = obtener_bloques_por_dev()
    for dev, bloques_list in bloques.items():
        print(f"📅 {dev}: {len(bloques_list)} bloque(s) ocupado(s)")
        for start, end in bloques_list:
            print(f"   🕓 {start.strftime('%Y-%m-%d %H:%M')} → {end.strftime('%H:%M')}")
        

🔎 Consultando reuniones de Luis Uran (luisuran@biamex.com)...
🔎 Consultando reuniones de Diego Martin Gogorza (diegogogorza@biamex.com)...
🔎 Consultando reuniones de Nicolas Pardo (nicolaspardo@biamex.com)...
🔎 Consultando reuniones de Martin Horn (martinhorn@biamex.com)...
🔎 Consultando reuniones de Facundo Capua (facundocapua@biamex.com)...
🔎 Consultando reuniones de Franco Lorenzo (francolorenzo@biamex.com)...
🔎 Consultando reuniones de Alan Mori - Carestino (alanmori@biamex.com)...
🔎 Consultando reuniones de Gastón Ojeda (gastonojeda@biamex.com)...
🔎 Consultando reuniones de Miguel Armentano (miguelarmentano@biamex.com)...
🔎 Consultando reuniones de Juan Ignacio Morelis - Carestino (juanmorelis@biamex.com)...
📅 Luis Uran: 10 bloque(s) ocupado(s)
   🕓 2025-08-18 08:30 → 09:00
   🕓 2025-08-19 08:30 → 09:00
   🕓 2025-08-20 08:30 → 09:00
   🕓 2025-08-20 09:00 → 09:30
   🕓 2025-08-21 08:30 → 09:00
   🕓 2025-08-22 08:30 → 09:00
   🕓 2025-08-25 08:30 → 09:00
   🕓 2025-08-26 08:30 → 09:00


In [5]:
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict


# === LOAD DATA ===
with open("jira_it_issues.json", "r", encoding="utf-8") as f:
    issues = json.load(f)["issues"]

with open("epics_due_lookup.json", "r", encoding="utf-8") as f:
    epic_due_lookup = json.load(f)

scheduled = []
tasks_by_dev = {}
issue_map = {}
graph = nx.DiGraph()

from datetime import timedelta

def contar_dias_laborables(start_date, end_date):
    dias = 0
    current = start_date
    while current <= end_date:
        if current.weekday() < 5:  # Lunes a Viernes
            dias += 1
        current += timedelta(days=1)
    return dias

for issue in issues:
    f = issue["fields"]
    status = f.get("status", {}).get("name", "")
    assignee = f.get("assignee")
    if not assignee:
        continue
    dev = assignee["displayName"]
    report = f.get("reporter")
    if not report:
        continue
    reporter = report["displayName"]
    key = issue["key"]
    summary = f.get("summary", "")
    module_info = f.get("customfield_10212")
    module_value = module_info["value"] if module_info else None
    estimate_hours = f.get("customfield_10608")

    if estimate_hours is None:
        estimate_hours = f.get("customfield_10016")

    if estimate_hours is None:
        estimate_seconds = f.get("timetracking", {}).get("originalEstimateSeconds") or f.get("aggregatetimeoriginalestimate")
        estimate_hours = estimate_seconds / 3600 if estimate_seconds else 0
    
    parent_key = f.get("parent", {}).get("key")
    epic_obj = epic_due_lookup.get(parent_key) if parent_key else None
    epic_due_str = epic_obj.get("due_date") if epic_obj else None
    due_str = f.get("duedate") or epic_due_str
    try:
        due_date = datetime.strptime(due_str, "%Y-%m-%d") if due_str else datetime(2100, 1, 1)
    except:
        due_date = datetime(2100, 1, 1)

    cf10442_raw = f.get("customfield_10442") or []
    cf10442_names = [u.get("displayName") for u in cf10442_raw if u.get("displayName")]
    suggested_devs = MODULE_DEVS.get(module_value, []) if module_value else []
    suggested_users = list(set(suggested_devs + cf10442_names))

    if dev in suggested_users:
        suggested_users.remove(dev)

    can_be_delegated = dev not in suggested_users and bool(suggested_users)
    delegation_note = f"🚨 Puede derivarse a: {', '.join(suggested_users)}" if can_be_delegated else ""



    
    
    task = {
        "key": key,
        "summary": summary,
        "estimate_hours": estimate_hours,
        "due_date": due_date,
        "assignee": dev,
        "status": status,
        "due_reason": None,
        "module": module_value,
        "suggested_users": suggested_users,
        "has_epic": bool(parent_key),
        "reporter": reporter
    }
    # 🔧 Nuevo: guardar claves de tareas que bloquean esta
    
    issue_map[key] = task

    
    if status in VALID_STATUSES:
        tasks_by_dev.setdefault(dev, []).append(task)
    graph.add_node(key)
    

    # for key, task in issue_map.items():
    #     blocker_due_dates = [
    #         issue_map[b]["due_date"]
    #         for b in graph.predecessors(key)
    #         if b in issue_map and issue_map[b]["due_date"]
    #     ]

    #     if blocker_due_dates:
    #         max_blocker_due = min(blocker_due_dates)
    #         if task["due_date"] < max_blocker_due:
    #             task["due_date_original"] = task["due_date"]
    #             task["due_date"] = max_blocker_due

    for link in f.get("issuelinks", []):
        if "inwardIssue" in link and link["type"]["name"] == "Blocks":
            graph.add_edge(link["inwardIssue"]["key"], key)
        elif "outwardIssue" in link and link["type"]["name"] == "Blocks":
            graph.add_edge(key, link["outwardIssue"]["key"])

    blockers = [src for src, tgt in graph.edges() if tgt == key]
    task["blockers"] = blockers

for key, task in issue_map.items():
    # Para cada tarea, mirar a quién bloquea
    blocked_keys = list(graph.successors(key))
    
    blocked_due_dates = [
        issue_map[b]["due_date"]
        for b in blocked_keys
        if b in issue_map and issue_map[b]["due_date"]
    ]

    blocker_key = [issue_map[b]["key"] for b in blocked_keys if b in issue_map and issue_map[b]["key"]]

    if blocked_due_dates:
        min_blocked_due = min(blocked_due_dates)
        suggested_due = min_blocked_due - timedelta(days=1)
        if task["due_date"] > suggested_due:
            task["due_date_original"] = task["due_date"]
            task["due_date"] = suggested_due
            task["note"] = f"🕓 Fecha de vencimiento ajustada por bloqueo a tarjeta {blocker_key} que vence el: {suggested_due.strftime('%Y-%m-%d')}"


# BLOQUES POR DEV (simulados)
bloques_por_dev = bloques
MINIMUM_SLOT_HOURS = 0.25
# === PLANIFICADOR POR DESARROLLADOR ===
def to_naive(dt):
    return dt.replace(tzinfo=None) if dt.tzinfo else dt

def planificar_tareas_para_dev(dev, tasks, bloques_ocupados):
    current_time = datetime.combine(DEFAULT_START_DATE.date(), datetime.strptime("08:30", "%H:%M").time())
    hours_per_day = DAILY_HOURS.get(dev, DAILY_HOURS["Default"])
    schedule = {}
    plan = []

    try:
        sorted_keys = list(nx.topological_sort(graph.subgraph([t["key"] for t in tasks])))
    except nx.NetworkXUnfeasible:
        sorted_keys = [t["key"] for t in tasks]
    sorted_keys.sort(key=lambda k: issue_map[k]["due_date"])

    def to_naive(dt):
        return dt.replace(tzinfo=None) if dt.tzinfo else dt
    
    for start, end in sorted(bloques_ocupados):
            plan.append({
                "developer": dev,
                "key": f"REUNION-{start.strftime('%Y%m%d%H%M')}",
                "summary": "⛔ Reunión",
                "has_epic": False,
                "due_date": start.date(),  # o end.date()
                "start": to_naive(start),
                "end": to_naive(end),
                "duration_hours": (to_naive(end) - to_naive(start)).total_seconds() / 3600,
                "type": "reunion"
            })

    for key in sorted_keys:
        task = issue_map[key]
        hours_left = task["estimate_hours"]
        start_time = None
       

        

        while hours_left > 0:
            date_str = current_time.strftime('%Y-%m-%d')
            used_today = schedule.get(date_str, 0)

            end_of_day = datetime.combine(current_time.date(), datetime.strptime("17:30", "%H:%M").time())
            available_time = (end_of_day - current_time).total_seconds() / 3600
            time_slot = min(available_time, hours_per_day - used_today, hours_left)

             # 🔧 NUEVO BLOQUE: limitar time_slot al hueco libre real antes del próximo conflicto
            next_conflict_start = None
            for start, end in sorted(bloques_ocupados):
                if to_naive(start) > to_naive(current_time):
                    next_conflict_start = to_naive(start)
                    break

            if next_conflict_start:
                gap_hours = (next_conflict_start - current_time).total_seconds() / 3600
                if gap_hours > 0:
                    time_slot = min(time_slot, gap_hours)

            if time_slot <= 0:
                next_day = current_time.date() + timedelta(days=1)
                while next_day.weekday() in [5, 6]:
                    next_day += timedelta(days=1)
                current_time = datetime.combine(next_day, datetime.strptime("08:30", "%H:%M").time())
                continue

            proposed_end = current_time + timedelta(hours=time_slot)

            bloque_conflictivo = any(
                to_naive(start) < to_naive(proposed_end) and to_naive(end) > to_naive(current_time)
                for start, end in bloques_ocupados
            )

            if bloque_conflictivo:
                conflictos = [
                    (start, end) for start, end in bloques_ocupados
                    if to_naive(start) < to_naive(proposed_end) and to_naive(end) > to_naive(current_time)
                ]
                conflictos.sort(key=lambda x: to_naive(x[0]))
                next_start = to_naive(conflictos[0][1])
                current_time = next_start
                continue

            if start_time is None:
                start_time = current_time

            if bloque_conflictivo:
                conflictos = [
                    (start, end) for start, end in bloques_ocupados
                    if to_naive(start) < to_naive(proposed_end) and to_naive(end) > to_naive(current_time)
                ]
                conflictos.sort(key=lambda x: to_naive(x[0]))
                next_start = to_naive(conflictos[0][1])
                current_time = next_start
                continue

            if start_time is None:
                start_time = current_time

            end_time = current_time + timedelta(hours=time_slot)
            plan.append({
                "developer": dev,
                "key": key,
                "summary": task["summary"],
                "has_epic": task["has_epic"],
                "due_date": task["due_date"],
                "start": current_time,
                "end": end_time,
                "duration_hours": time_slot,
                "suggested_users": task.get("suggested_users", []),
                "blockers": task.get("blockers", []),
                "note": task.get("note", ""),
                "reporter": task.get("reporter", "")
            })
            

            schedule[date_str] = used_today + time_slot
            hours_left -= time_slot
            current_time = end_time

    return plan

# === PLANIFICACIÓN INICIAL ===
# for dev in tasks_by_dev:
#     scheduled += planificar_tareas_para_dev(dev, tasks_by_dev[dev], bloques_por_dev[dev])

# === REBALANCEO ===
planned_hours_by_dev = defaultdict(float)
project_hours_by_dev = defaultdict(float)

for s in scheduled:
    if s.get("type") == "reunion":
        continue
    planned_hours_by_dev[s["developer"]] += s["duration_hours"]
    if issue_map[s["key"]]["has_epic"]:
        project_hours_by_dev[s["developer"]] += s["duration_hours"]

for dev in tasks_by_dev:
    all_tasks = tasks_by_dev[dev]
    
    within_range = [t for t in all_tasks if DEFAULT_START_DATE <= t["due_date"] <= DEFAULT_END_DATE]
    epic_tasks_in_range = sorted([t for t in within_range if t["has_epic"]], key=lambda t: t["due_date"])
    non_epic_tasks_in_range = sorted([t for t in within_range if not t["has_epic"]], key=lambda t: t["due_date"], reverse=True)
    epic_tasks_out_of_range = sorted(
        [t for t in all_tasks if t["has_epic"] and t not in within_range],
        key=lambda t: t["due_date"]
    )

    total_in_range = sum(t["estimate_hours"] for t in within_range)
    current_epic = sum(t["estimate_hours"] for t in epic_tasks_in_range)
    non_epic_hours = sum(t["estimate_hours"] for t in non_epic_tasks_in_range)
    min_ratio = MIN_PROJECT_RATIO.get(dev, MIN_PROJECT_RATIO["Default"])

    print(f"\n🔎 {dev}:")
    print(f"  🎯 Epic target: {min_ratio * 100:.1f}%")
    print(f"  📊 Total in range: {total_in_range:.1f} h | Epic in range: {current_epic:.1f} h")
    print(f"  🔍 Epic tasks out of range: {len(epic_tasks_out_of_range)}")
    for t in epic_tasks_out_of_range:
        print(f"    - {t['key']} ({t['estimate_hours']} h) due {t['due_date'].strftime('%Y-%m-%d')}")

    # 🟨 Forzar tareas con épica fuera de rango hasta cumplir el mínimo
    # horas_planificables_en_rango = (
    #         sum(t["estimate_hours"] for t in epic_tasks_in_range) +
    #         sum(t["estimate_hours"] for t in non_epic_tasks_in_range) +
    #         sum(t["estimate_hours"] for t in epic_tasks_out_of_range)
    #     )
    
    dias_laborables = contar_dias_laborables(DEFAULT_START_DATE, DEFAULT_END_DATE)
    horas_por_dia = DAILY_HOURS.get(dev, DAILY_HOURS["Default"])
    horas_planificables_en_rango = dias_laborables * horas_por_dia
    while epic_tasks_out_of_range:
        task = epic_tasks_out_of_range.pop(0)
        task_estimate = task["estimate_hours"]

        # Moverla dentro del rango
        task["due_date_original"] = task["due_date"]
        print(f" modificando {task['key']} due {t['due_date'].strftime('%Y-%m-%d')}")
        task["due_date"] = DEFAULT_START_DATE
        epic_tasks_in_range.append(task)
        current_epic += task_estimate

        # Recalcular ratio después de agregarla
        ratio_actual = current_epic / horas_planificables_en_rango if horas_planificables_en_rango > 0 else 0
        print(f"✔️ Ratio tras forzar {task['key']}: {current_epic:.1f} / ({horas_planificables_en_rango:.1f}) = {ratio_actual * 100:.1f}%")
        
        
        
        if ratio_actual >= min_ratio:
            break

    # ❌ Eliminar tareas sin épica si sigue sin cumplirse el mínimo
    ratio_actual = current_epic / (current_epic + non_epic_hours) if (current_epic + non_epic_hours) > 0 else 0
    while non_epic_tasks_in_range and ratio_actual < min_ratio:
        removed = non_epic_tasks_in_range.pop()
        non_epic_hours -= removed["estimate_hours"]
        ratio_actual = current_epic / (current_epic + non_epic_hours) if (current_epic + non_epic_hours) > 0 else 0

    # 🔄 Reordenar: primero épicas, luego tareas normales
    balanced_tasks = sorted(epic_tasks_in_range, key=lambda t: t["due_date"]) + \
                     sorted(non_epic_tasks_in_range, key=lambda t: t["due_date"])
    leftovers = [t for t in all_tasks if t not in balanced_tasks]
    tasks_by_dev[dev] = balanced_tasks + leftovers

    # 🔁 Replanificar solo ese desarrollador
    scheduled = [s for s in scheduled if s["developer"] != dev]
    scheduled += planificar_tareas_para_dev(dev, tasks_by_dev[dev], bloques_por_dev[dev])


# === RESUMEN FINAL ===
scheduled_by_dev_and_key = defaultdict(lambda: defaultdict(list))
for s in scheduled:
    scheduled_by_dev_and_key[s["developer"]][s["key"]].append(s)

print("\n📝 Developer Task Schedule Summary:\n")
for dev, tasks in tasks_by_dev.items():
    print(f"👨‍💻 Developer: {dev}")
    try:
        sorted_keys = list(nx.topological_sort(graph.subgraph([t["key"] for t in tasks])))
    except nx.NetworkXUnfeasible:
        sorted_keys = [t["key"] for t in tasks]
    sorted_keys.sort(key=lambda k: issue_map[k]["due_date"])
    for key in sorted_keys:
        task = issue_map[key]
        scheds = sorted(scheduled_by_dev_and_key[dev].get(key, []), key=lambda x: x["start"])
        if not scheds:
            continue
        start_time = scheds[0]["start"]
        end_time = scheds[-1]["end"]
        total_hours = sum(s["duration_hours"] for s in scheds)
        note_lines = []
        if "note" in task and task["note"]:
            note_lines.append(task["note"])

        # 🚨 Sugerencias de derivación
        if task.get("suggested_users"):
            note_lines.append(f"🚨 Puede derivarse a: {', '.join(task['suggested_users'])}")

        # 📛 Bloqueadores
        if task.get("blockers"):
            blocker_msgs = []
            for blocker_key in task["blockers"]:
                blocker = issue_map.get(blocker_key)
                if blocker:
                    blocker_msgs.append(f"{blocker_key}: {blocker['summary']} ({blocker['assignee']}, Due: {blocker['due_date'].strftime('%Y-%m-%d')})")
                else:
                    blocker_msgs.append(blocker_key)
            note_lines.append(f"📛 Debido a bloqueo de: " + " | ".join(blocker_msgs))

         # Agregar nota personalizada si existe
        if "note" in task and task["note"]:
            note_lines.append(task["note"])

        note = "\n     " + "\n     ".join(note_lines) if note_lines else ""
        print(
            f"   🔹 {key}: {round(total_hours, 1)}h → Due: {task['due_date'].strftime('%Y-%m-%d')} | "
            f"Finish: Start: {start_time.strftime('%Y-%m-%d %H:%M')} | End: {end_time.strftime('%Y-%m-%d %H:%M')}{note}"
        )
    print()


🔎 Facundo Capua:
  🎯 Epic target: 10.0%
  📊 Total in range: 0.0 h | Epic in range: 0.0 h
  🔍 Epic tasks out of range: 2
    - IT-17547 (24.0 h) due 2100-01-01
    - IT-15195 (24.0 h) due 2100-01-01
 modificando IT-17547 due 2100-01-01
✔️ Ratio tras forzar IT-17547: 24.0 / (80.0) = 30.0%

🔎 Luis Uran:
  🎯 Epic target: 50.0%
  📊 Total in range: 4.0 h | Epic in range: 0.0 h
  🔍 Epic tasks out of range: 22
    - IT-19532 (2.0 h) due 2025-04-11
    - IT-19694 (2.0 h) due 2025-05-15
    - IT-20471 (10.0 h) due 2025-08-15
    - IT-16647 (2.0 h) due 2025-12-31
    - IT-16646 (2.0 h) due 2025-12-31
    - IT-16645 (3.0 h) due 2025-12-31
    - IT-16644 (2.0 h) due 2025-12-31
    - IT-16643 (2.0 h) due 2025-12-31
    - IT-16642 (4.0 h) due 2025-12-31
    - IT-16641 (3.0 h) due 2025-12-31
    - IT-16640 (4.0 h) due 2025-12-31
    - IT-16639 (3.0 h) due 2025-12-31
    - IT-16638 (7.0 h) due 2025-12-31
    - IT-16635 (3.0 h) due 2025-12-31
    - IT-16634 (6.0 h) due 2025-12-31
    - IT-16633 (5.0 h)

In [6]:
from collections import defaultdict



# Cálculo de horas trabajadas con y sin épica por desarrollador SOLO en la ventana de 11 días
epic_stats = defaultdict(lambda: {"con_epica": 0.0, "sin_epica": 0.0})

for s in scheduled:
    dev = s["developer"]
    key = s["key"]
    start = s["start"]

    # Solo incluir tareas dentro del rango
    if not (DEFAULT_START_DATE <= start <= DEFAULT_END_DATE):
        continue

    task = issue_map.get(key)
    if not task:
        continue

    estimate = s["duration_hours"]
    parent_key = issues[[i["key"] for i in issues].index(key)]["fields"].get("parent", {}).get("key")

    if parent_key:
        epic_stats[dev]["con_epica"] += estimate
    else:
        epic_stats[dev]["sin_epica"] += estimate

# Mostrar resultados
print(f"\n📊 Estadísticas de horas (entre {DEFAULT_START_DATE.strftime('%Y-%m-%d')} y {DEFAULT_END_DATE.strftime('%Y-%m-%d')}):\n")

for dev, stats in epic_stats.items():
    con_epica = stats["con_epica"]
    sin_epica = stats["sin_epica"]
    total = con_epica + sin_epica
    pct_con = (con_epica / total * 100) if total > 0 else 0
    pct_sin = (sin_epica / total * 100) if total > 0 else 0

    print(f"👨‍💻 {dev}")
    print(f"   🟨 Con épica: {con_epica:.1f} h ({pct_con:.1f}%)")
    print(f"   🟦 Sin épica: {sin_epica:.1f} h ({pct_sin:.1f}%)")
    print(f"   📌 Total: {total:.1f} h\n")


📊 Estadísticas de horas (entre 2025-08-18 y 2025-08-29):

👨‍💻 Facundo Capua
   🟨 Con épica: 24.0 h (33.3%)
   🟦 Sin épica: 48.0 h (66.7%)
   📌 Total: 72.0 h

👨‍💻 Luis Uran
   🟨 Con épica: 2.0 h (3.7%)
   🟦 Sin épica: 52.0 h (96.3%)
   📌 Total: 54.0 h

👨‍💻 Gastón Ojeda
   🟨 Con épica: 5.0 h (11.1%)
   🟦 Sin épica: 40.0 h (88.9%)
   📌 Total: 45.0 h

👨‍💻 Franco Lorenzo
   🟨 Con épica: 28.0 h (70.0%)
   🟦 Sin épica: 12.0 h (30.0%)
   📌 Total: 40.0 h

👨‍💻 Alan Mori - Carestino
   🟨 Con épica: 31.5 h (63.6%)
   🟦 Sin épica: 18.0 h (36.4%)
   📌 Total: 49.5 h

👨‍💻 Juan Ignacio Morelis - Carestino
   🟨 Con épica: 4.0 h (66.7%)
   🟦 Sin épica: 2.0 h (33.3%)
   📌 Total: 6.0 h

👨‍💻 Miguel Armentano
   🟨 Con épica: 45.0 h (100.0%)
   🟦 Sin épica: 0.0 h (0.0%)
   📌 Total: 45.0 h

👨‍💻 Nicolas Pardo
   🟨 Con épica: 60.5 h (100.0%)
   🟦 Sin épica: 0.0 h (0.0%)
   📌 Total: 60.5 h



In [7]:
with open("planificacion.json", "w") as f:
    json.dump(scheduled, f, default=str, indent=2)